In [1]:
import re
import string
from math import sqrt
import pickle
import time

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

In [2]:
# Đọc dữ liệu
items_df = pd.read_csv('./items.csv')
ratings_df = pd.read_csv('./ratings.csv')
users_df = pd.read_csv('./users.csv')

In [3]:
ratings_matrix = ratings_df.to_numpy()

def get_items_rated_by_user(rate_matrix, user_id):
    user_row_indices = np.where(rate_matrix[:, 0] == user_id + 1)[0]
    if len(user_row_indices) == 0:
        return np.array([]), np.array([])  

    item_ids = rate_matrix[user_row_indices, 1] - 1  
    scores = rate_matrix[user_row_indices, 2]
    return item_ids.astype(int), scores  

# TfidfVectorizer và cosine similarity
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words="english")
vectors = tfidf_vectorizer.fit_transform(items_df["Description"]).toarray()
similarity = cosine_similarity(vectors)

# Chuyển ratings_df thành numpy array cho việc truy xuất
ratings_matrix = ratings_df[['CustomerID', 'stockCodeTransform', 'rate']].to_numpy()

n_items = len(items_df)
n_users = len(users_df)

In [4]:

# Chia dữ liệu thành train và test (80:20)
train_matrix, test_matrix = train_test_split(ratings_matrix, test_size=0.2, random_state=42)


In [5]:
# Sử dụng mô hình KNN để huấn luyện trên tập train
n_items = len(items_df)
n_users = len(users_df)
k_neighbors = 5  

knn_models = [None] * n_users

for n in range(n_users):
    ids, scores = get_items_rated_by_user(train_matrix, n)

    if len(ids) == 0:
        knn_models[n] = None  # Nếu người dùng chưa đánh giá phim nào
        continue
    
    k_neighbors_user = min(k_neighbors, len(ids))  # Điều chỉnh số lượng hàng xóm
    
    knn = KNeighborsRegressor(n_neighbors=k_neighbors_user, weights='uniform', metric='cosine')
    
    Xhat = vectors[ids, :]  # Đặc trưng của các phim đã được đánh giá
    y = scores  # Điểm đánh giá thực tế
    
    knn.fit(Xhat, y)
    
    knn_models[n] = knn


In [6]:
# Hàm tính RMSE
def evaluate(Yhat, rates):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        if len(ids) == 0:
            continue  # Nếu người dùng chưa đánh giá phim nào, bỏ qua
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred 
        se += (e * e).sum()
        cnt += e.size 
    return sqrt(se / cnt) if cnt != 0 else float('nan')

In [7]:
# Dự đoán trên toàn bộ ma trận Yhat cho tập huấn luyện và kiểm tra
Yhat_train = np.zeros((n_items, n_users))
Yhat_test = np.zeros((n_items, n_users))

for n in range(n_users):
    if knn_models[n] is not None:
        Yhat_train[:, n] = knn_models[n].predict(vectors)  # Dự đoán cho tập huấn luyện
        Yhat_test[:, n] = knn_models[n].predict(vectors)   # Dự đoán cho tập kiểm tra
    else:
        Yhat_train[:, n] = 0  # Nếu người dùng chưa đánh giá phim nào
        Yhat_test[:, n] = 0

# Tính RMSE cho tập huấn luyện và kiểm tra
train_rmse = evaluate(Yhat_train, train_matrix)
test_rmse = evaluate(Yhat_test, test_matrix)

# Làm tròn RMSE
train_rmse_rounded = round(train_rmse, 4) 
test_rmse_rounded = round(test_rmse, 4)

print(f"RMSE trên tập huấn luyện: {train_rmse_rounded}")
print(f"RMSE trên tập kiểm tra: {test_rmse_rounded}")

RMSE trên tập huấn luyện: 0.8377
RMSE trên tập kiểm tra: 1.0378


In [8]:
import pickle

# Lưu toàn bộ mô hình KNN cho tất cả người dùng vào một tệp duy nhất
with open('knn_models.pkl', 'wb') as file:
    pickle.dump(knn_models, file)
    
# Tải lại toàn bộ mô hình KNN từ tệp
with open('knn_models.pkl', 'rb') as file:
    knn_models = pickle.load(file)


In [9]:
# Tạo tập dữ liệu đầy đủ 
full_data  = ratings_matrix 

In [10]:
# Dự đoán trên toàn bộ ma trận Yhat cho tập huấn luyện và kiểm tra
Yhat_CB = np.zeros((n_items, n_users))

for n in range(n_users):
    if knn_models[n] is not None:
        Yhat_CB[:, n] = knn_models[n].predict(vectors)  # Dự đoán cho tập huấn luyện
    else:
        Yhat_CB[:, n] = 0  # Nếu người dùng chưa đánh giá phim nào

In [11]:
# Lưu ma trận Yhat vào file numpy
np.save('Yhat_CB.npy', Yhat_CB)

# Để tải lại ma trận Yhat sau này
Yhat_loaded = np.load('Yhat_CB.npy')
print(Yhat_loaded)
Yhat_loaded.shape[0]

[[4.4 5.  4.6 ... 3.6 2.2 5. ]
 [4.4 5.  4.6 ... 3.6 2.2 5. ]
 [4.4 5.  4.6 ... 3.6 2.2 5. ]
 ...
 [4.4 5.  4.6 ... 3.6 2.2 4.8]
 [4.4 5.  4.6 ... 3.6 2.2 4.8]
 [4.4 5.  4.6 ... 3.6 2.2 4.8]]


3642